sometimes code is too fast for browser driver. in such cases, try increasing sleep time.

In [1]:
from bs4 import BeautifulSoup as bs 
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
import pandas as pd

In [2]:
DRIVER_PATH = 'C:/Users/thisi/Documents/GitHub/pq-scraper/chromedriver_win32/chromedriver.exe'

In [3]:
def element_completely_viewable(driver, elem):
    elem_left_bound = elem.location.get('x')
    elem_top_bound = elem.location.get('y')
    elem_width = elem.size.get('width')
    elem_height = elem.size.get('height')
    elem_right_bound = elem_left_bound + elem_width
    elem_lower_bound = elem_top_bound + elem_height

    win_upper_bound = driver.execute_script('return window.pageYOffset')
    win_left_bound = driver.execute_script('return window.pageXOffset')
    win_width = driver.execute_script('return document.documentElement.clientWidth')
    win_height = driver.execute_script('return document.documentElement.clientHeight')
    win_right_bound = win_left_bound + win_width
    win_lower_bound = win_upper_bound + win_height

    return all((win_left_bound <= elem_left_bound,
                win_right_bound >= elem_right_bound,
                win_upper_bound <= elem_top_bound,
                win_lower_bound >= elem_lower_bound)
              )

def find_and_click(driver, elem):
    y_height = 0
    while not element_completely_viewable(driver, elem):
        driver.execute_script(f"window.scrollTo(0, {y_height})") 
        y_height += 100
        time.sleep(0.5)
    elem.click()

driver = webdriver.Chrome(DRIVER_PATH) 
 
page_url = 'https://www.parliament.gov.sg/history/list-of-mps-by-parliament' 
driver.get(page_url) 
driver.maximize_window() 
driver.implicitly_wait(20) 
time.sleep(2) 

parliament_combobox = driver.find_element(
    by=By.ID,
    value='Parliament'
)

num_parliaments = int(parliament_combobox.get_attribute('childElementCount'))
print(f'Total of {num_parliaments} parliaments')


C:\Users\thisi\AppData\Local\Temp\ipykernel_3680\1172286970.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


Total of 14 parliaments


In [4]:
mps = []

for parliament_index in range(1, num_parliaments + 1):
    # need this line of code to prevent stale element reference. not sure why but just keep it. 
    parliament_combobox = driver.find_element(
        by=By.ID,
        value='Parliament'
    )

    current_parliament_option = driver.find_element(
        by=By.XPATH,
        value=f'//*[@id="Parliament"]/option[{parliament_index}]'
    )
    
    find_and_click(driver, parliament_combobox)
    time.sleep(0.5)
    find_and_click(driver, current_parliament_option)
    time.sleep(2)
    
    print(f'Looking at parliament {parliament_index}')
    
    full_table = driver.find_element(
        by=By.XPATH,
        value='/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul'
    )
    
    num_rows = int(full_table.get_attribute('childElementCount'))
    print(f'Found {num_rows} MPs')
    
    mps_this_parliament = []
    
    for i in range(1, num_rows+1):
        name_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[1]/div[2]'
        )
        
        party_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[2]'
        )
        
        legis_assembly_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[3]'
        )
        
        also_in_div = driver.find_element(
            by=By.XPATH,
            value=f'/html/body/form/div[3]/div[2]/div[2]/div[1]/div[3]/ul/li[{i}]/div/div[4]'
        )
        
        name, party, legis_assembly, also_in = list(map(
            lambda div: div.get_attribute('innerHTML').strip(),
            [name_div, party_div, legis_assembly_div, also_in_div]
        ))
        
        other_parl_nums = []
        
        if legis_assembly:
            other_parl_nums.append(0) # 0 denotes legislative assembly
        
        for other_parl in also_in.split(', '):
            other_parl_num = int(other_parl[:-2])
            other_parl_nums.append(other_parl_num)
        
        mps_this_parliament.append((name, party, tuple(other_parl_nums)))
    
    mps += mps_this_parliament

Looking at parliament 1
Found 64 MPs
Looking at parliament 2
Found 64 MPs
Looking at parliament 3
Found 65 MPs
Looking at parliament 4
Found 78 MPs
Looking at parliament 5
Found 76 MPs
Looking at parliament 6
Found 79 MPs
Looking at parliament 7
Found 84 MPs
Looking at parliament 8
Found 92 MPs
Looking at parliament 9
Found 99 MPs
Looking at parliament 10
Found 103 MPs
Looking at parliament 11
Found 103 MPs
Looking at parliament 12
Found 110 MPs
Looking at parliament 13
Found 111 MPs
Looking at parliament 14
Found 102 MPs


In [5]:
print(f'{len(mps)} entries')
print(f'{len(set(mps))} unique enetries')

1230 entries
452 unique enetries


In [6]:
dataframe = pd.DataFrame(set(mps), columns=['Name','Party','Parliaments'])
dataframe

,Name,Party,Parliaments
0,Ho Tat Kin,People's Action Party,"(6, 7, 8, 9)"
1,Mary Liew,Nominated Member of Parliament,"(12,)"
2,Kow Kee Seng,Barisan Sosialis,"(1,)"
3,Anthea Ong,Nominated Member of Parliament,"(13,)"
4,Zaqy Mohamad,People’s Action Party,"(11, 12, 13, 14)"
...,...,...,...
447,Tan Wu Meng,People's Action Party,"(13, 14)"
448,Ng Yeow Chong,People's Action Party,"(1, 2, 3, 4)"
449,Teo Hup Teck,People's Action Party,"(1,)"
450,Ow Chin Hock,People's Action Party,"(4, 5, 6, 7, 8, 9)"


In [7]:
dataframe.to_csv('mps.csv')